### Lab 2 .bil file

In [3]:
import requests
import json
import pprint
import zipfile 
import urllib
#import arcpy
import pandas as pd
from urllib.parse import urlencode
from urllib.parse import urlparse, parse_qsl

In [28]:
import ftplib

## Download Data

In [39]:
# Use FTP to connect to "prism.nacse.org" and login use email address with anonymous password
ftp = ftplib.FTP("prism.nacse.org")
ftp.login("anonymous","zhux0474@umn.edu")


'230 Anonymous access granted, restrictions apply'

In [79]:
# change directories and navigate the file tree to get the files 
ftp.cwd("/normals_4km/ppt")
fnbase="PRISM_ppt_30yr_normal_4kmM2_annual"
fnend = "_bil.zip"

In [80]:
fn = fnbase + fnend
with open(fn, "wb") as file:
    #ftp.retrbinary(f"RETR {fn}", file.write) 
    ftp.retrbinary("RETR "+fn, file.write) 
ftp.quit()

'221 Goodbye.'

In [82]:
with zipfile.ZipFile('PRISM_ppt_30yr_normal_4kmM2_annual_bil.zip','r') as zip_ref:
    zip_ref.extractall('Data')

## Create Space Time Cube

### Convert bil to tiff

In [5]:
arcpy.conversion.RasterToOtherFormat(r"C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_01_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_02_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_03_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_04_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_05_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_06_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_07_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_08_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_09_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_10_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_11_bil.bil;C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\PRISM_ppt_30yr_normal_4kmM2_all_bil\PRISM_ppt_30yr_normal_4kmM2_12_bil.bil", r"C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\Tifff", "TIFF")

<Result ''>

### Create Mosaic Dataset

In [6]:
arcpy.management.CreateMosaicDataset(r"C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\MyProject\MyProject.gdb", "ppt", "PROJCS['WGS_1984_Web_Mercator_Auxiliary_Sphere',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Mercator_Auxiliary_Sphere'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],PARAMETER['Auxiliary_Sphere_Type',0.0],UNIT['Meter',1.0]]", None, '', "NONE", None)

<Result 'C:\\Users\\zhux0474\\Desktop\\Lab2.2\\Lab2.2\\MyProject\\MyProject.gdb\\APPT'>

### Add Tiff to Mosaic Dataset

In [7]:
arcpy.management.AddRastersToMosaicDataset("ppt", "Raster Dataset", r"C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\Tifff", "UPDATE_CELL_SIZES", "UPDATE_BOUNDARY", "NO_OVERVIEWS", None, 0, 1500, None, '', "SUBFOLDERS", "ALLOW_DUPLICATES", "NO_PYRAMIDS", "NO_STATISTICS", "NO_THUMBNAILS", '', "NO_FORCE_SPATIAL_REFERENCE", "NO_STATISTICS", None, "NO_PIXEL_CACHE", r"C:\Users\zhux0474\AppData\Local\ESRI\rasterproxies\APPT")

<Result 'APPT'>

### Add Timestamp Field

In [8]:
arcpy.management.CalculateField(r"ppt\Footprint", "Timestamp", "DateAdd(Date(2020,0,1),$feature.OBJECTID-1,'month')", "ARCADE", '', "TEXT")

<Result 'APPT\\Footprint'>

In [10]:
arcpy.management.CalculateField(r"ppt\Footprint", "Variable", '"ppt"', "PYTHON3", '', "TEXT")

<Result 'APPT\\Footprint'>

### Build Multidimensional Info

In [14]:
arcpy.md.BuildMultidimensionalInfo("ppt", "Variable", "Timestamp # #", "ppt # #")

<Result 'APPT'>

### Make Multidimensional Raster 

In [17]:
arcpy.md.MakeMultidimensionalRasterLayer("ppt", "ppt_Mlayer", "ppt", "ALL", None, None, '', '', '', None, '', "-125.020833333 24.0625000000001 -66.479166667 49.9375000000001", "DIMENSIONS")

<Result 'ppt_Mlayer'>

### Create Space Time Cube from Multidimensional Raster Layer

In [4]:
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer("ppt_Mlayer", r"C:\Users\zhux0474\Desktop\Lab2.2\Lab2.2\MyProject\cube3.nc", "ZEROS")

<Result 'C:\\Users\\zhux0474\\Desktop\\Lab2.2\\Lab2.2\\MyProject\\cube3.nc'>